In [ ]:
from google.colab import drive

# Unmount the drive first
drive.flush_and_unmount()
print('Drive unmounted')

# Remove existing files from the mountpoint if it exists
import os
if os.path.exists('/content/drive'):
  !rm -rf '/content/drive'  # Use with caution! This permanently deletes all files in the directory.
  print('Files removed from mountpoint')

# Remount the drive
drive.mount('/content/drive')
print('Drive mounted')

Drive not mounted, so nothing to flush and unmount.
Drive unmounted
Mounted at /content/drive
Drive mounted


In [ ]:
path = '/content/drive/Shareddrives/DrunkKiddo'

In [ ]:
import tensorflow as tf # importing tensorflow
from tensorflow.keras.applications import MobileNetV2 #this is the model we will be building off of
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D # I have no clue what this does, but thank his excellency, ChatGPT for this
from tensorflow.keras.models import Model #Once again, Lord GPT's suggestion

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) # this imports out model
#let us explain a little bit about this:
# MoveNetV2 is our big, beutiful, light-weight model
#we set include_top = False because we want to remove that very last layer of our network, because we do not care about all of the various classifications
#that already exist for our data set
#finally the input_shape(224, 224, 3) refers to the input image size, as well as the number of pixel types used (3, since RGB)

In [ ]:
import os
import shutil

# Define the source directory and the target directories
source_dir = "/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images"
sober_dir = "/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/sober"
drunk_dir = "/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/drunk"

# Create target directories if they don't exist
os.makedirs(sober_dir, exist_ok=True)
os.makedirs(drunk_dir, exist_ok=True)

# Iterate through the images in the source directory
for filename in os.listdir(source_dir):
    source_file = os.path.join(source_dir, filename)
    if filename.startswith("Soberface"):
        target_file = os.path.join(sober_dir, filename)
    elif filename.startswith("Drunkface"):
        target_file = os.path.join(drunk_dir, filename)
    else:
        continue

    try:
        shutil.move(source_file, target_file)
    except FileNotFoundError:
        print(f"File not found: {source_file}")

print("Images have been separated into 'sober' and 'drunk' directories.")

Images have been separated into 'sober' and 'drunk' directories.


In [ ]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split

# Define the source directories for sober and drunk images
sober_dir = "/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/sober"
drunk_dir = "/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/drunk"

# Create lists of all images
sober_images = [os.path.join(sober_dir, img) for img in os.listdir(sober_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]
drunk_images = [os.path.join(drunk_dir, img) for img in os.listdir(drunk_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

# Combine the lists and create labels
all_images = sober_images + drunk_images
labels = [0] * len(sober_images) + [1] * len(drunk_images)  # 0 for sober, 1 for drunk

# Shuffle the data
combined = list(zip(all_images, labels))
random.shuffle(combined)
all_images, labels = zip(*combined)

# Split the data into training, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(all_images, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Define the target directories for train, validation, and test sets
train_dir = "/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/train"
val_dir = "/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/val"
test_dir = "/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/test"

# Create target directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Function to copy images to their respective directories
def copy_images(images, labels, target_dir):
    for img, label in zip(images, labels):
        label_dir = "sober" if label == 0 else "drunk"
        os.makedirs(os.path.join(target_dir, label_dir), exist_ok=True)
        shutil.copy(img, os.path.join(target_dir, label_dir, os.path.basename(img)))

# Copy images to train, validation, and test directories
copy_images(train_images, train_labels, train_dir)
copy_images(val_images, val_labels, val_dir)
copy_images(test_images, test_labels, test_dir)

print("Data has been shuffled and split into train, validation, and test sets.")

Data has been shuffled and split into train, validation, and test sets.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Directories for training, validation, and testing datasets
train_dir = "/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/train"
val_dir = "/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/val"
test_dir = "/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/test"

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only rescaling for validation and testing
val_test_datagen = ImageDataGenerator(rescale=1.0/255)

# Generating training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Generating validation data
val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Generating test data
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Building the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

# Evaluating the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

Found 2992 images belonging to 2 classes.
Found 998 images belonging to 2 classes.
Found 998 images belonging to 2 classes.
Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 225s 2s/step - accuracy: 0.5847 - loss: 0.7848 - val_accuracy: 0.7465 - val_loss: 0.5116
Epoch 2/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 213s 2s/step - accuracy: 0.7280 - loss: 0.5419 - val_accuracy: 0.7285 - val_loss: 0.5576
Epoch 3/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 220s 2s/step - accuracy: 0.7721 - loss: 0.4898 - val_accuracy: 0.7906 - val_loss: 0.4525
Epoch 4/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 210s 2s/step - accuracy: 0.8139 - loss: 0.4191 - val_accuracy: 0.7946 - val_loss: 0.4394
Epoch 5/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 213s 2s/step - accuracy: 0.8144 - loss: 0.4159 - val_accuracy: 0.8116 - val_loss: 0.4357
Epoch 6/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.8201 - loss: 0.4174 - val_accuracy: 0.8246 - val_loss: 0.4077
Epoch 7/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 258s 2s/step - accuracy: 0.8392 - loss: 0.3635 - val_accuracy: 0.8327 - val_

In [ ]:
import os

# Create the directory if it doesn't exist
os.makedirs('/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/saved_model/', exist_ok=True)

# Save the entire model to a HDF5 file
model.save('/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/model.h5')

# For saving the model after training in the recommended Keras format
model.save('/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/saved_model/my_model.keras')

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/model.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load MobileNetV2 and freeze its layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # Ensure 'output' is a KerasTensor

# Integrate with your existing model
stronger_model = Model(inputs=base_model.input, outputs=output)
stronger_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only rescaling for validation and testing
val_test_datagen = ImageDataGenerator(rescale=1.0/255)

# Generating training data
train_generator = train_datagen.flow_from_directory(
    '/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Generating validation data
val_generator = val_test_datagen.flow_from_directory(
    '/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/images/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Train the stronger model
stronger_model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

# Save the stronger model in Keras format
stronger_model.save('/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/saved_model/stronger_model.keras')

Found 2992 images belonging to 2 classes.
Found 998 images belonging to 2 classes.
Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 226s 2s/step - accuracy: 0.8345 - loss: 0.3645 - val_accuracy: 0.9138 - val_loss: 0.2063
Epoch 2/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.9306 - loss: 0.1907 - val_accuracy: 0.9289 - val_loss: 0.1803
Epoch 3/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 225s 2s/step - accuracy: 0.9397 - loss: 0.1552 - val_accuracy: 0.9168 - val_loss: 0.1982
Epoch 4/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 216s 2s/step - accuracy: 0.9599 - loss: 0.1078 - val_accuracy: 0.9409 - val_loss: 0.1498
Epoch 5/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 266s 2s/step - accuracy: 0.9658 - loss: 0.0917 - val_accuracy: 0.9349 - val_loss: 0.1561
Epoch 6/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 256s 3s/step - accuracy: 0.9692 - loss: 0.0922 - val_accuracy: 0.9489 - val_loss: 0.1290
Epoch 7/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.9640 - loss: 0.0867 - val_accuracy: 0.9559 - val_loss: 0.1031
Epoch 8/20
94/94 ━━━━━━━━━━━

In [ ]:
from tensorflow.keras.models import load_model
import json

# Load the Keras model
model = load_model('/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/saved_model/stronger_model.keras')

# Convert the model to JSON format
model_json = model.to_json()

# Save the JSON to a file
with open('/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/saved_model/stronger_model.json', 'w') as json_file:
    json_file.write(model_json)

# Save the weights to a bin file
weights = model.get_weights()
with open('/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/saved_model/stronger_model_weights.bin', 'wb') as bin_file:
    for layer_weights in weights:
        bin_file.write(layer_weights.tobytes())

In [13]:
!ngrok authtoken 2tdCy3lohz9eStVvQLnJT4BFCQu_2JWFyZoPMeSUqzTaaBUtQ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [14]:
import gradio as gr
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import numpy as np
from pyngrok import ngrok

# Load the Keras model
model = load_model('/content/drive/Shareddrives/DrunkKiddo/UJInebriate-main/saved_model/stronger_model.keras')

def prepare_image(image, target_size):
    image = image.resize(target_size)  # Resize the image
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    return image

def predict_image(image):
    image = prepare_image(image, target_size=(224, 224))
    prediction = model.predict(image)[0][0]
    result = "Not Drunk" if prediction > 0.5 else "Drunk"
    return result

# Create a Gradio interface
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),  # Use PIL type to ensure compatibility with .resize()
    outputs="text",
    title="Drunk Detection",
    description="Upload an image to predict if the person is drunk or not."
)

# Start Gradio interface and get the public URL using ngrok
public_url = ngrok.connect(7860)
print(f'Public URL: {public_url}')
iface.launch()

Public URL: NgrokTunnel: "https://f60f-34-60-85-19.ngrok-free.app" -> "http://localhost:7860"
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://da1177674f7e91bc5a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
